<a target="_blank" href="https://colab.research.google.com/github/wbfrench1/barker_DATA606/blob/main/src/Create_spaCy_NER_model_dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Followed this tutorial
https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch5/04_NER_using_spaCy%20-%20CoNLL.ipynb#scrollTo=X4wBa1MGwTwy

In [37]:
import os
from google.colab import drive
import pandas as pd
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)
import numpy as np
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/wbfrench1/barker_DATA606/main/data/movie_data_word_pos_iob_w_space_sep.iob', sep=' ')
print(df.shape)
df.head()

(322035, 4)


,Sentence #,Word,POS,Tag
0,1,what,WP,o
1,1,movies,NNS,o
2,1,star,VBP,o
3,1,bruce,NN,b-actor
4,1,willis,NN,i-actor


#### Add periods to the end of each sentence

<p>In the test data at https://github.com/explosion/spaCy/blob/master/extra/example_data/ner_example_data/ner-token-per-line-with-pos.iob   it appears that each sentence should have a period at the end of it.</p>

In [39]:
# select indices where the label in Sentence # is != to the
## previous Sentence
# https://stackoverflow.com/questions/72726217/get-index-of-row-where-column-value-changes-from-previous-row
l_indices = df.index[df['Sentence #'].diff().eq(1)]

In [40]:
# https://stackoverflow.com/questions/15888648/is-it-possible-to-insert-a-row-at-an-arbitrary-position-in-a-dataframe-using-pan
# get an index location between each sentence 
l_new_indicies = l_indices - .5
l_new_indicies

Float64Index([     4.5,     13.5,     22.5,     35.5,     46.5,     53.5,     62.5,     71.5,     81.5,     89.5,
              ...
              321827.5, 321842.5, 321859.5, 321880.5, 321904.5, 321923.5, 321968.5, 321974.5, 321995.5, 322016.5],
             dtype='float64', length=21986)

In [81]:
for sent_num, f_index in enumerate(l_new_indicies):
    df.loc[f_index] = sent_num + 1, '.', '.', 'o'
    if sent_num % 1000 == 0:
        print(sent_num)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000


In [82]:
# sort the indices and reset the index
df1 = df.loc[df.index.sort_values()].copy()
df1 = df1.reset_index().drop(columns='index').copy()

In [83]:
df1.head(20)

,Sentence #,Word,POS,Tag
0,1,what,WP,o
1,1,movies,NNS,o
2,1,star,VBP,o
3,1,bruce,NN,b-actor
4,1,willis,NN,i-actor
5,1,.,.,o
6,2,show,VB,o
7,2,me,PRP,o
8,2,films,NNS,o
9,2,with,IN,o


#  Does the data need to be randomized?
<p>I have movie trivia questions and iob labels.  The sentence structure and context gives the words and their labels meaning.  So I believe that it should not be randomized.</p>

<p>Question:  Beyond ensuring that the component words of each sentence stay together, do I need to ensure that the model only looks to complete sentences since one question is unrelated to the next? </p>

In [84]:
# since groups of words in a sentence should stay together, I must divide the 
## training/test/validation data into sets according at the sentence level

# Get the max number of sentences
df1['Sentence #'].max()

21987

In [85]:
df1['Tag'] = df1['Tag'].str.upper()

In [86]:
#https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test

# get an array of 1 to n questions
a_q_num = np.arange(1,21988)

In [87]:
#divvy the questions up into groups.  Since questions should stay together
## randomization is not an issue.
l_a_indices_of_train_test_val_splits = np.split(a_q_num, [int(.6 * len(a_q_num)), int(.8 * len(a_q_num))])
l_a_indices_of_train_test_val_splits

[array([    1,     2,     3, ..., 13190, 13191, 13192]),
 array([13193, 13194, 13195, ..., 17587, 17588, 17589]),
 array([17590, 17591, 17592, ..., 21985, 21986, 21987])]

In [88]:
df_train = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[0])].copy()
df_test = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[1])].copy()
df_val = df1.loc[df1['Sentence #'].isin(l_a_indices_of_train_test_val_splits[2])].copy()

print('df1_train.shape:', df_train.shape)
print('df1_test.shape:', df_test.shape)
print('df1_val.shape:', df_val.shape)

df1_train.shape: (157234, 4)
df1_test.shape: (93714, 4)
df1_val.shape: (93073, 4)


In [89]:
#https://stackoverflow.com/questions/31247198/python-pandas-write-content-of-dataframe-into-text-file
l_movie_iob_csv_datasets= ['train_movie_iob', 'test_movie_iob', 'val_movie_iob']

for str_data_set in l_movie_iob_csv_datasets:
    # save dataset as a csv
    df_train[ ['Word', 'POS', 'Tag']].to_csv(str_data_set,sep="\t", index=False, header=False)
    # removes the column headers from the file
    with open(str_data_set, 'w') as f:
        dfAsString = df_train[ ['Word', 'POS', 'Tag']].to_string(header=False, index=False)
        f.write(dfAsString)

In [90]:
!head 'val_movie_iob' -n 12 

                   what   WP              O
                 movies  NNS              O
                   star  VBP              O
                  bruce   NN        B-ACTOR
                 willis   NN        I-ACTOR
                      .    .              O
                   show   VB              O
                     me  PRP              O
                  films  NNS              O
                   with   IN              O
                   drew  NNS        B-ACTOR
              barrymore   RB        I-ACTOR


In [94]:
#Read the IOB data from content folder, and store the formatted data into a folder spacyNER_data

# !mkdir spacyNER_data
if 'spacyNER_data' not in os.listdir():
    os.mkdir('spacyNER_data')

#the above lines create folder if it doesn't exist.
#!python -m spacy convert  "train_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 
!python -m spacy convert  "test_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 
#!python -m spacy convert  "val_movie_iob" spacyNER_data -c ner  -s -n 10 -b en_core_web_sm 

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-04 18:19:40.578206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 18:19:41.473234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 18:19:41.473322: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [98]:
!python -m spacy train en_core_web_sm model spacyNER_data/train_movie_iob.spacy spacyNER_data/test_movie_iob.spacy -G -p tagger,ner

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-04 18:26:12.274948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 18:26:13.160046: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 18:26:13.160141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [78]:
!echo "BEFORE : (test_movie_iob)"
!head "train_movie_iob" -n 11 | tail -n 9


BEFORE : (test_movie_iob)
                   star  VBP              O
                  bruce   NN        B-ACTOR
                 willis   NN        I-ACTOR
                                           
                   show   VB              O
                     me  PRP              O
                  films  NNS              O
                   with   IN              O
                   drew  NNS        B-ACTOR


In [79]:
try:
    !echo "AFTER : (spacyNER_data/train_movie_iob.spacy)"
    !head "spacyNER_data/train_movie_iob.spacy"-n 11 


SyntaxError: ignored